In [2]:
!pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [3]:
# Surpress warnings from using older version of sklearn:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from datetime import datetime


#Load the Data

In [5]:
file_path ='/content/KPMG_dataset.xlsx'

In [6]:
# Read the XLSX file into a DataFrame
df = pd.read_excel(file_path)

In [7]:
# List the sheet names in the XLSX file
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
sheet_names

['Title Sheet',
 'Transactions',
 'NewCustomerList',
 'CustomerDemographic',
 'CustomerAddress']

In [8]:
# Read a specific sheet into a DataFrame
customer_demographic = pd.read_excel(file_path, sheet_name='CustomerDemographic', skiprows=1)

In [9]:
# Read a specific sheet into a DataFrame
customer_addresses = pd.read_excel(file_path, sheet_name='CustomerAddress', skiprows=1)

In [10]:
transactions = pd.read_excel(file_path, sheet_name='Transactions', skiprows=1)

In [11]:
# Display the first few rows of the DataFrame to verify it was read correctly
customer_demographic.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [12]:
customer_addresses.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [13]:
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


#EDA of Customer Demographics

## 1. Data Completeness

In [14]:
customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [15]:
customer_demographic.describe()

,customer_id,past_3_years_bike_related_purchases,tenure
count,4000.000000,4000.000000,3913.000000
mean,2000.500000,48.890000,10.657041
std,1154.844867,28.715005,5.660146
min,1.000000,0.000000,1.000000
25%,1000.750000,24.000000,6.000000
50%,2000.500000,48.000000,11.000000
75%,3000.250000,73.000000,15.000000
max,4000.000000,99.000000,22.000000


In [16]:
# Check for missing values
missing_values = customer_demographic.isnull().sum()
missing_values

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

In [70]:
customer_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [71]:
customer_addresses.describe()

,customer_id,postcode,property_valuation
count,3999.000000,3999.000000,3999.000000
mean,2003.987997,2985.755939,7.514379
std,1154.576912,844.878364,2.824663
min,1.000000,2000.000000,1.000000
25%,1004.500000,2200.000000,6.000000
50%,2004.000000,2768.000000,8.000000
75%,3003.500000,3750.000000,10.000000
max,4003.000000,4883.000000,12.000000


In [72]:
missing_values = customer_addresses.isnull().sum()

# Display the missing values count for each column
print("Missing Values:")
print(missing_values)

Missing Values:
customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64


## 2. Data Accuracy

In [17]:
# Check for data types
data_types = customer_demographic.dtypes
data_types

customer_id                                     int64
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
default                                        object
owns_car                                       object
tenure                                        float64
dtype: object

In [18]:
customer_demographic.nunique()

customer_id                            4000
first_name                             3139
last_name                              3725
gender                                    6
past_3_years_bike_related_purchases     100
DOB                                    3448
job_title                               195
job_industry_category                     9
wealth_segment                            3
deceased_indicator                        2
default                                  90
owns_car                                  2
tenure                                   22
dtype: int64

In [19]:
# Check for duplicate customer IDs
duplicate_ids = customer_demographic['customer_id'].duplicated()
duplicate_count = duplicate_ids.sum()
print("Duplicate Customer IDs:", duplicate_count)

Duplicate Customer IDs: 0


There are no duplicate customer_ids, meaning each ID is unique

In [20]:
# Check for outliers or unexpected values
# You can define a range of valid customer IDs based on your business rules
valid_customer_id_range = (min(customer_demographic['customer_id']), max(customer_demographic['customer_id']))

print("Valid Customer ID Range:", valid_customer_id_range)

Valid Customer ID Range: (1, 4000)


all customer IDs in the dataset fall within this range, which means there are no outliers in terms of customer IDs

In [21]:
# Check for unexpected characters in first names and last names
unexpected_characters_first_name = customer_demographic[customer_demographic['first_name'].str.isalpha()]['first_name']
#unexpected_characters_last_name = customer_demographic[customer_demographic['last_name'].str.isalpha()]['last_name']

# Check for inconsistencies, such as titles or non-name entries
inconsistent_first_names = customer_demographic[customer_demographic['first_name'].str.contains('[0-9]')]
#inconsistent_last_names = df[df['last_name'].str.contains('[0-9]')]

# Display the results
print("First Names with Unexpected Characters:")
print(unexpected_characters_first_name)

print("\nLast Names with Unexpected Characters:")
#print(unexpected_characters_last_name)

print("\nInconsistent First Names (Containing Numbers):")
print(inconsistent_first_names[['customer_id', 'first_name']])

print("\nInconsistent Last Names (Containing Numbers):")
#print(inconsistent_last_names[['customer_id', 'last_name']])

First Names with Unexpected Characters:
0         Laraine
1             Eli
2           Arlin
3          Talbot
5            Curr
          ...    
3995      Rosalia
3996       Blanch
3997       Sarene
3998    Patrizius
3999        Kippy
Name: first_name, Length: 3983, dtype: object

Last Names with Unexpected Characters:

Inconsistent First Names (Containing Numbers):
Empty DataFrame
Columns: [customer_id, first_name]
Index: []

Inconsistent Last Names (Containing Numbers):


There are some first and last names that contain non-alphabetical characters that are not typical in a person's name. Examples of unexpected characters might include numbers, special symbols, or punctuation marks.

In [22]:
customer_demographic['gender'].unique().tolist()

['F', 'Male', 'Female', 'U', 'Femal', 'M']

In [23]:
# Extract the 'Past 3 Years Bike-Related Purchases' column
purchase_counts = customer_demographic['past_3_years_bike_related_purchases']

# Check for negative values
negative_values = purchase_counts[purchase_counts < 0]

# Check for unrealistic outliers (you can set a threshold as needed)
unrealistic_outliers = purchase_counts[purchase_counts > 100]  # Adjust the threshold as appropriate

# Display the results
print("Negative Purchase Counts:")
print(negative_values)

print("\nUnrealistic Outliers:")
print(unrealistic_outliers)

Negative Purchase Counts:
Series([], Name: past_3_years_bike_related_purchases, dtype: int64)

Unrealistic Outliers:
Series([], Name: past_3_years_bike_related_purchases, dtype: int64)


 There are no negative purchase counts or unrealistic outliers in the "Past 3 Years Bike-Related Purchases" column

In [24]:
# Calculate the reference date (current date)
current_date = datetime.now()

In [25]:
# Define a reasonable date range for DOB (e.g., last 80 years)
min_dob = current_date - pd.DateOffset(years=80)

In [26]:
# Check for unrealistic DOB values
unrealistic_dob = customer_demographic[customer_demographic['DOB'] > current_date]  # DOB in the future
unrealistic_dob = unrealistic_dob.append(customer_demographic[customer_demographic['DOB'] < min_dob])  # DOB too far in the past

# Print unrealistic DOB values
print("Unrealistic DOB Values:")
print(unrealistic_dob[['customer_id', 'DOB']])

Unrealistic DOB Values:
      customer_id        DOB
33             34 1843-12-21
719           720 1931-10-23
1091         1092 1935-08-22
2412         2413 1943-08-11
3409         3410 1940-09-22


In [28]:
missing_dob =  customer_demographic[ customer_demographic['DOB'].isnull()]
missing_dob

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
143,144,Jory,Barrabeale,U,71,NaT,Environmental Tech,IT,Mass Customer,N,NaN,No,NaN
167,168,Reggie,Broggetti,U,8,NaT,General Manager,IT,Affluent Customer,N,NaN,Yes,NaN
266,267,Edgar,Buckler,U,53,NaT,NaN,IT,High Net Worth,N,NaN,No,NaN
289,290,Giorgio,Kevane,U,42,NaT,Senior Sales Associate,IT,Mass Customer,N,NaN,No,NaN
450,451,Marlow,Flowerdew,U,37,NaT,Quality Control Specialist,IT,High Net Worth,N,NaN,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778,3779,Ulick,Daspar,U,68,NaT,NaN,IT,Affluent Customer,N,NaN,No,NaN
3882,3883,Nissa,Conrad,U,35,NaT,Legal Assistant,IT,Mass Customer,N,NaN,No,NaN
3930,3931,Kylie,Epine,U,19,NaT,NaN,IT,High Net Worth,N,NaN,Yes,NaN
3934,3935,Teodor,Alfonsini,U,72,NaT,NaN,IT,High Net Worth,N,NaN,Yes,NaN


In [30]:
# Extract the 'job_title' column
job_titles = customer_demographic['job_title']

# Initialize lists to store findings
spelling_variations = []
abbreviations = []
non_standard_titles = []
special_characters = []

# Define a list of standard job titles or keywords
standard_job_titles = ["Manager", "Engineer", "Analyst", "Consultant", "Sales", "Teacher", "Doctor", "Nurse", "Technician"]

# Loop through job titles to identify patterns and inconsistencies
for title in job_titles:
    # Check if the title is a string (not NaN or float)
    if isinstance(title, str):
        # Spelling Variations
        if "Engineer" in title and "Eng." in title:
            spelling_variations.append(title)

        # Abbreviations
        if "Mgr." in title:
            abbreviations.append(title)

        # Non-standard Titles
        if title not in standard_job_titles:
            non_standard_titles.append(title)

        # Special Characters
        if not title.isalpha():
            special_characters.append(title)

# Display the findings
print("Job Titles with Spelling Variations:")
print(spelling_variations)

print("\nJob Titles with Abbreviations:")
print(abbreviations)

print("\nNon-Standard Job Titles:")
print(non_standard_titles)

print("\nJob Titles with Special Characters:")
print(special_characters)


Job Titles with Spelling Variations:
[]

Job Titles with Abbreviations:
[]

Non-Standard Job Titles:
['Executive Secretary', 'Administrative Officer', 'Recruiting Manager', 'Senior Editor', 'Media Manager I', 'Business Systems Development Analyst', 'Senior Quality Engineer', 'Nuclear Power Engineer', 'Developer I', 'Account Executive', 'Junior Executive', 'Media Manager IV', 'Sales Associate', 'Professor', 'Geological Engineer', 'Project Manager', 'Safety Technician I', 'Research Assistant I', 'Accounting Assistant III', 'Editor', 'Research Nurse', 'Senior Quality Engineer', 'Safety Technician III', 'Staff Accountant III', 'Legal Assistant', 'Product Engineer', 'Legal Assistant', 'Information Systems Manager', 'VP Quality Control', 'Social Worker', 'Senior Cost Accountant', 'Assistant Media Planner', 'Payment Adjustment Coordinator', 'Food Chemist', 'Accountant III', 'Director of Sales', 'Assistant Media Planner', 'Senior Financial Analyst', 'Registered Nurse', 'Biostatistician II', 'V

- This dataset includes job titles that do not conform to common industry roles or standard job titles. Examples include "Budget/Accounting Analyst I," "Nurse Practicioner," "Chief Design Engineer," and many others.

- This dataset includes job titles that contain special characters, numbers, or symbols not typical in job titles. Examples include "Media Manager I," "Biostatistician IV," and "Community Outreach Specialist." Depending on your data requirements, you may need to assess whether these titles are acceptable or if they should be standardized.

In [31]:
# Assuming your DataFrame is named 'df'
unique_categories = customer_demographic['job_industry_category'].unique()
print(unique_categories)

['Health' 'Financial Services' 'Property' 'IT' nan 'Retail' 'Argiculture'
 'Manufacturing' 'Telecommunications' 'Entertainment']


In [32]:
expected_categories = ["Health", "Financial Services", "Property", "IT", "Retail", "Agriculture", "Manufacturing", "Telecommunications", "Entertainment"]

In [33]:
unexpected_categories = [category for category in unique_categories if category not in expected_categories]
print("Unexpected Categories:", unexpected_categories)

Unexpected Categories: [nan, 'Argiculture']


In [34]:
category_counts = customer_demographic['job_industry_category'].value_counts()
print("Category Counts:\n", category_counts)


Category Counts:
 Manufacturing         799
Financial Services    774
Health                602
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: job_industry_category, dtype: int64


"Manufacturing" and "Financial Services" are the most common industry categories among your customers.

In [37]:
# Assuming 'wealth_segment' is the column of interest
wealth_segment_column = customer_demographic['wealth_segment']

# 1. Check Unique Values:
unique_values = wealth_segment_column.unique()
print(unique_values)

['Mass Customer' 'Affluent Customer' 'High Net Worth']


In [39]:
expected_categories = ["Mass Customer", "High Net Worth", "Affluent Customer"]  # List of expected categories

# Check if unique values match the expected categories
unexpected_values = [value for value in unique_values if value not in expected_categories]

if unexpected_values:
    print("Unexpected Wealth Segment Values:")
    print(unexpected_values)
else:
    print("Wealth segment values match the expected categories.")

Wealth segment values match the expected categories.


In [40]:
# 2. Check for Consistency:
segment_counts = wealth_segment_column.value_counts()

# Display the counts of each wealth segment
print("\nWealth Segment Counts:")
print(segment_counts)


Wealth Segment Counts:
Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64


The "wealth_segment" column appears to have consistent and expected values. The counts align with the categories you mentioned earlier ("Mass Customer," "High Net Worth," and "Affluent Customer"). The counts show how many customers fall into each wealth segment category, providing valuable insights into the distribution of customer wealth segments within the dataset.

In [42]:
# Check unique values in the 'deceased_indicator' column
unique_deceased_values = customer_demographic['deceased_indicator'].unique()
print(unique_deceased_values)

['N' 'Y']


In [45]:
# Ensure that only 'Y' and 'N' are valid values for the 'deceased_indicator' column
valid_values = ['Y', 'N']

# Check for inconsistent values
inconsistent_values = [value for value in unique_deceased_values if value not in valid_values]

if inconsistent_values:
    # Print and handle inconsistent values
    print("Inconsistent values found in 'deceased_indicator':", inconsistent_values)

else:
    # No inconsistent values found, the data is consistent
    print("No inconsistent values found in 'deceased_indicator' column.")

No inconsistent values found in 'deceased_indicator' column.


In [47]:
# Examine unique values in the 'Default' column
unique_defaults =customer_demographic['default'].unique()
print(unique_defaults)

['"\'' "<script>alert('hi')</script>" datetime.datetime(2018, 2, 1, 0, 0)
 '() { _; } >_[$($())] { touch /tmp/blns.shellshock2.fail; }' 'NIL'
 'ðµ ð ð ð' 'â°â´âµâââ' '(â¯Â°â¡Â°ï¼â¯ï¸µ â»ââ»)' '0/0' 'ð©ð½'
 'ÅâÂ´Â®â\xa0Â¥Â¨ËÃ¸Ïââ' 'nil' -100 'â°â´âµ' 'ð'
 1000000000000000049861653971908893017010268485438462151574892930611988399099305815384459015356416
 'ï¾ï½¥â¿ã¾â²(ï½¡ââ¿âï½¡)â±â¿ï½¥ï¾' 'Î©âÃ§ââ«ËÂµâ¤â¥Ã·' 'ÅâÂ´â°ËÃÂ¨ËÃâââ'
 'ï¼ï¼ï¼' '../../../../../../../../../../../etc/hosts'
 '×Ö¸×Ö°×ªÖ¸×testØ§ÙØµÙØ\xadØ§Øª Ø§ÙØªÙØ\xadÙÙ' '<>?:"{}|_+' '\'\'\'\'"'
 ",./;'[]\\-=" '() { 0; }; touch /tmp/blns.shellshock1.fail;'
 'ì¬íê³¼íì ì´íì°êµ¬ì' 'testâ\xa0testâ«'
 '0ï¸â£ 1ï¸â£ 2ï¸â£ 3ï¸â£ 4ï¸â£ 5ï¸â£ 6ï¸â£ 7ï¸â£ 8ï¸â£ 9ï¸â£ ð' nan
 '!@#$%^&*()' "'"
 'Ì¦HÍÌ¬Ì¤ÌÌ¤eÍ ÍÌÌ¥ÌÌ»ÍÌwÌhÌÌ¯ÍoÌÍÌÍÌ±Ì® ÒÌºÌÌÌÍWÌ·Ì¼Ì\xadaÌºÌªÍiÌ¨ÍÍÌ\xadÍÌ¯ÌtÌ¶Ì¼Ì®sÌÌÍÍ Ì\xa0Ì«Ì\xa0BÌ»ÍÍÍÍÌ³eÌµhÌµÌ¬ÍÌ«ÍiÌÌ¹ÍÌ³Ì³Ì®ÍÌ«nÍdÌ´ÌªÌÌ ÍÌ°ÍÌ©ÍÍÍÌ²TÍ¢ÍÌ¼ÍÌªhÍÍÌ®Ì»eÌ¬ÌÍÌ Ì¤Ì¹ÌWÍÍÍÌÌÍÍaÍÍÍÌ¹Ì¼'
 '../../../../../../../../../../../etc/passwd%00' 

The output for the unique values in the "Default" column appears to contain a wide range of unexpected and non-standard values, including special characters, symbols, and unusual text strings. This suggests that there may be data quality issues or data entry errors in this column.

In [49]:
# Check unique values in the "owns_car" column
unique_values = customer_demographic['owns_car'].unique()
print(unique_values)

['Yes' 'No']


In [50]:
# Check for inconsistencies (non-"Yes" or "No" values)
inconsistent_values = [value for value in unique_values if value not in ["Yes", "No"]]

# Print inconsistent values
if inconsistent_values:
    print("Inconsistent 'owns_car' values:", inconsistent_values)
else:
    print("All 'owns_car' values are consistent.")

All 'owns_car' values are consistent.


In [51]:
# Check for missing values in the "owns_car" column
missing_values = customer_demographic['owns_car'].isnull().sum()
print("Missing 'owns_car' values:", missing_values)

Missing 'owns_car' values: 0


In [54]:
# Check for negative tenure values
negative_tenure = customer_demographic[customer_demographic['tenure'] < 0]

# Check for unrealistic tenure values (you can define a threshold)
unrealistic_tenure = customer_demographic[customer_demographic['tenure'] > 100]  # Adjust the threshold as needed

# Display the rows with negative tenure values (if any)
print("Rows with Negative Tenure:")
print(negative_tenure)

# Display the rows with unrealistic tenure values (if any)
print("\nRows with Unrealistic Tenure:")
print(unrealistic_tenure)

Rows with Negative Tenure:
Empty DataFrame
Columns: [customer_id, first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, default, owns_car, tenure]
Index: []

Rows with Unrealistic Tenure:
Empty DataFrame
Columns: [customer_id, first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, default, owns_car, tenure]
Index: []


In [73]:
# Check for data types
data_types = customer_addresses.dtypes
data_types

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

In [77]:
customer_addresses.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [74]:
def is_valid_state(state):
    valid_states = ['NY', 'CA', 'TX', ...]  # Example valid state abbreviations or names

    # Check if the state value is in the list of valid states
    return state in valid_states

# Apply the validation function to the "state" column
df['valid_state'] = df['state'].apply(is_valid_state)



TypeError: ignored

## 3. Data Redundancy

In [55]:
# Check for duplicate customer IDs
duplicate_customer_ids = customer_demographic[customer_demographic.duplicated(subset=['customer_id'], keep=False)]

# Display duplicate records (if any)
if not duplicate_customer_ids.empty:
    print("Duplicate Customer IDs:")
    print(duplicate_customer_ids)
else:
    print("No Duplicate Customer IDs Found")

No Duplicate Customer IDs Found


There are no duplicate records.

## 4. Data Integrity

In [57]:
# Use df.isnull().sum() to count missing values in each column
missing_values = customer_demographic.isnull().sum()

# Print the results to see the extent of missing data in each column
print("Missing Values in Each Column:")
print(missing_values)

Missing Values in Each Column:
customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64


## 4. Data Currency

In [66]:
# Define your reasonable tenure range
reasonable_tenure_range = (0, 20)  # Adjust based on your business history

# Filter rows where tenure is outside the reasonable range
tenure_outside_range = customer_demographic[(customer_demographic['tenure'] < reasonable_tenure_range[0]) | (customer_demographic['tenure'] > reasonable_tenure_range[1])]

# Display the customers with extreme tenure values
print("Customers with extreme tenure values:")
tenure_outside_range


Customers with extreme tenure values:


,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,unusual_job_title,unusual_industry_category
24,25,Geoff,Assaf,Male,72,1976-12-02,Accounting Assistant III,Financial Services,Mass Customer,N,×Ö¸×Ö°×ªÖ¸×testØ§ÙØµÙØ­Ø§Øª Ø§ÙØªÙØ­ÙÙ,Yes,21.0,False,False
27,28,Fee,Zellmer,Male,5,1973-09-30,Senior Quality Engineer,Health,Affluent Customer,N,",./;'[]\-=",Yes,21.0,False,False
35,36,Lurette,Stonnell,Female,33,1977-11-09,VP Quality Control,NaN,Affluent Customer,N,"<>?:""{}|_+",No,22.0,False,False
38,39,Hunfredo,Smalley,Male,5,1979-04-16,Assistant Media Planner,Entertainment,Mass Customer,N,',No,22.0,False,False
61,62,Sorcha,Roggers,Female,38,1979-07-06,NaN,IT,Mass Customer,N,(ï¾à²¥çà²¥ï¼ï¾ï»¿ â»ââ»,Yes,22.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,3912,Ryon,Darridon,Male,89,1986-10-27,Social Worker,Health,Affluent Customer,N,1,No,22.0,False,False
3913,3914,Izabel,Roelofs,Female,79,1988-05-21,Assistant Manager,Entertainment,Affluent Customer,N,ï½ï½¨(Â´âï½â©,Yes,22.0,False,False
3929,3930,Blancha,Baldi,Female,43,1988-01-06,NaN,Financial Services,High Net Worth,N,(â¯Â°â¡Â°ï¼â¯ï¸µ â»ââ»),No,22.0,False,False
3967,3968,Alexandra,Kroch,Female,99,1977-12-22,NaN,Property,High Net Worth,N,ï¾ï½¥â¿ã¾â²(ï½¡ââ¿âï½¡)â±â¿ï½¥ï¾,No,22.0,False,False


Assuming Sprocket has been in existence for 20 years, there are customer records with tenure values above 20. These records represent customers who have been associated with your company for a longer period than expected based on your business's history

In [69]:
# Check for customers marked as deceased
deceased_customers = customer_demographic[customer_demographic['deceased_indicator'] == 'Y']

# Print the list of deceased customers
print("List of Deceased Customers:")
deceased_customers

# Count the number of deceased customers
num_deceased_customers = deceased_customers.shape[0]
print(f"Total Number of Deceased Customers:")
num_deceased_customers

List of Deceased Customers:
Total Number of Deceased Customers:


2

The presence of deceased customers in the dataset has implications for data currency. There are 2 customers marked as deceased, suggesting that there are individuals in the dataset who are no longer alive, according to the recorded information.

### 1.2 Data Cleaning

In [ ]:
EDA of Customer Address

#EDA of Customer Demographics

In [ ]:
customer_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


#EDA of Transactions

In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n